In [1]:
import import_ipynb### install this pakage first
import group12 as g12### import first model,put the file under the same folder 
import group3 as g3### import second model,put the file under the same folder
import numpy as np
import pygame
import time
#import test1

importing Jupyter notebook from group12.ipynb
pygame 2.0.2 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
importing Jupyter notebook from group3.ipynb


In [2]:
#competition_list=[g3,g5,g12]

In [3]:
def update_by_man(event,mat):
    """
    This function detects the mouse click on the game window. Update the state matrix of the game. 
    input: 
        event:pygame event, which are either quit or mouse click)
        mat: 2D matrix represents the state of the game
    output:
        mat: updated matrix
    """
    done=False
    if event.type==pygame.QUIT:
        done=True
    if event.type==pygame.MOUSEBUTTONDOWN:
        (x,y)=event.pos
        row = round((y - 40) / 40)     
        col = round((x - 40) / 40)
        mat[row][col]=1
    return mat, done
def draw_board(screen, size):
    screen.fill((230, 185, 70))
    for x in range(size):
        pygame.draw.line(screen, [0, 0, 0], [25 + 50 * x, 25], [25 + 50 * x, size*50-25], 1)
        pygame.draw.line(screen, [0, 0, 0], [25, 25 + 50 * x], [size*50-25, 25 + 50 * x], 1)
    pygame.display.update()


def update_board(screen, state):
    indices = np.where(state != 0)
    for (row, col) in list(zip(indices[0], indices[1])):
        if state[row][col] == 1:
            pygame.draw.circle(screen, [0, 0, 0], [25 + 50 * col, 25 + 50 * row], 15, 15)
        elif state[row][col] == -1:
            pygame.draw.circle(screen, [255, 255, 255], [25 + 50 * col, 25 + 50 * row], 15, 15)
    pygame.display.update()
    
    
def draw_stone(screen, mat):
    """
    This functions draws the stones according to the mat. It draws a black circle for matrix element 1(human),
    it draws a white circle for matrix element -1 (computer)
    input:
        screen: game window, onto which the stones are drawn
        mat: 2D matrix representing the game state
    output:
        none
    """
    black_color = [0, 0, 0]
    white_color = [255, 255, 255]
    #M=len(mat)
    d=int(560/(M-1))
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if mat[i][j]==1:
                pos = [40+d * j, 40+d* i ]
                pygame.draw.circle(screen, black_color, pos, 18,0)
            elif mat[i][j]==-1:
                pos = [40+d* j , 40+d * i]
                pygame.draw.circle(screen, white_color, pos, 18,0)


def check_for_done(mat):
    """
    please write your own code testing if the game is over. Return a boolean variable done. If one of the players wins
    or the tie happens, return True. Otherwise return False. Print a message about the result of the game.
    input: 
        2D matrix representing the state of the game
    output:
        True,1:Black win the game
        True,0:Draw
        True,-1:White win the game
        False,0:Not complete
    """
    
    return done,result

def update_by_pc(mat):
    """
    This is the core of the game. Write your code to give the computer the intelligence to play a Five-in-a-Row game 
    with a human
    input:
        2D matrix representing the state of the game.
    output:
        2D matrix representing the updated state of the game.
    """
    return mat





# main function

In [4]:
import pygame
import numpy as np
def main():
    global M
    M = 8
    pygame.init()
    screen = pygame.display.set_mode((50*M, 50*M))
    pygame.display.set_caption('Interface of Five-in-a-Row')
    done = False
    pc_step = 0    
    mat=np.zeros((M,M),int)
    draw_board(screen, M)
    pygame.display.update()
    #mat[int(M/2)][int(M/2)]=1
    update_board(screen, mat)
    while not done:
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                done=True
                
            if event.type!=pygame.QUIT:
                
                ###it is g12 turn
                mat=g12.update_by_pc(mat)
                ###
                
                
                update_board(screen, mat)
                print("mat=",mat)
                if done:
                    print('winer is:',_)
                    break
                    
                    
                ### it is g3 turn
                mat = g3.update_by_pc(-1*mat)#transform to the result for g3 to put white stone
                ###
            
                mat=mat*(-1)
                done, _ = check_for_done(mat)
                print('now mat is',mat,_)                
                print('update2')
                # check for win or tie
                update_board(screen, mat)
                done, _ = check_for_done(mat)
                if done==True:
                    print('winer is:',_)
                    break


if __name__ == '__main__':
    main()
    


mat= [[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
Simulation times: 41 in 5 s.
now mat is [[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]] 0
update2
mat= [[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]
Simulation times: 46 in 5 s.
now mat is [[ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  1  0  1  0  0  0]
 [ 0  0  0 -1  1  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]] 0
update2
mat= [[ 0  0  0  0  0 

mat= [[ 0  0  0 -1  0  0  0  0]
 [ 0  1 -1 -1 -1  1  0  0]
 [ 0  0 -1  1  1 -1 -1  1]
 [-1 -1  1 -1  1  1  1 -1]
 [ 0  1  0 -1  1  1  1 -1]
 [ 0  0  0  1 -1  1 -1 -1]
 [ 0  0  1  0  1  1  0  0]
 [ 0 -1  0  0  0 -1  0  0]]
Simulation times: 109 in 5 s.
now mat is [[ 0  0  0 -1  0  0  0  0]
 [ 0  1 -1 -1 -1  1  0  0]
 [ 0  0 -1  1  1 -1 -1  1]
 [-1 -1  1 -1  1  1  1 -1]
 [ 0  1  0 -1  1  1  1 -1]
 [ 0  0  0  1 -1  1 -1 -1]
 [ 0  0  1  0  1  1  0  1]
 [ 0 -1  0  0  0 -1  0  0]] 0
update2


IndexError: index 8 is out of bounds for axis 1 with size 8